# Juliaの導入

Google ColabではPythonが標準となっていますが、今回の実習では、多くの計算が手軽に、より素早く、実行できるJuliaLangを使ってみましょう。

まず、以下のセルの命令を実行してみてください。

shitf+enterもしくは左側のボタンで実行可能です。

こちらのやり方は、以下の記事で紹介されたものです:

https://qiita.com/ueuema/items/ca1b326f5df10a4203bd

https://qiita.com/cometscome_phys/items/1ba6ec181bb0fe1b35d5

In [ ]:
!curl -sSL "https://julialang-s3.julialang.org/bin/linux/x64/1.6/julia-1.6.2-linux-x86_64.tar.gz" -o julia.tar.gz
!tar -xzf julia.tar.gz -C /usr --strip-components 1
!rm -rf julia.tar.gz*
!julia -e 'using Pkg; pkg"add IJulia"'

## ランタイムの設定

上のセルの実行後、

上部のメニューから、"ランタイム"→"ランタイムのタイプを変更"を開いて、

ランタイムのタイプが"julia-1.6"になっているか確認し、"保存"を実行してください。

## Juliaの動作確認

以下のセルを実行してみてください。

**Juliaのversionが表示されたらJuliaの導入に成功です。**

In [ ]:
versioninfo()

# 計算の実例

実際にJuliaを用いたプログラムを動かしてみます。

プログラムは、以下のMcMillanの論文に基づき、液体ヘリウム４の２点動径分布関数g(r)を変分モンテカルロ法によって計算します。

[W. L. McMillan, Phys. Rev. 138, A442 (1965)](https://journals.aps.org/pr/abstract/10.1103/PhysRev.138.A442)


## 準備

手軽な計算のためにパッケージを導入します。

以下では計算データを図として書き出すためのPlotsと、

乱数を用いるためにRandomを準備しています。

"import Pkg"には数分かかることがあります。

In [ ]:
import Pkg; Pkg.add("Plots")
using Plots
using Random

## モジュール

以下では定数と関数を束ねたモジュールを作成しています。

In [ ]:
module Helium4

# physical constant
    hbar = 1.054571e-34 # kg m^2 / s
    kB = 1.380649e-23 # J / K
  # ヘリウム4原子の質量
    massHe4 = 4.00260325415 * 1.0e-3 / 6.022140e+23 # kg
    mass = kB * massHe4 * 1.0e-20 / hbar^2 # K^-1 angstrom^-2
  # レナード-ジョーンズポテンシャルのパラメータ
    epsLJ = 1.41102e-22 / kB # K
    sigmaLJ = 2.556 # angstrom

# パラメータと変数の集まり    
    mutable struct Param
        a1::Float64
        a2::Float64
        L::Float64
        N::Int
        rcut::Float64
        config::Array{Float64,2} # (3,N)
        dist::Array{Float64,2} #  (N,N)
        Param() = new()
    end

# 初期化
    function initialize(param::Param, a1, a2, L, N)
        param.a1 = a1
        param.a2 = a2
        param.L = L
        param.N = N
        param.rcut = 0.5*a1
        param.config = zeros(Float64,3,N) # column-major
        param.dist = zeros(Float64,N,N) # column-major
        return 0 
    end

    function initialize_config(param::Param)
        for j in 1:param.N
            for i in 1:3
                param.config[i,j] = param.L * rand(Float64)
            end
        end
        for iupdate in 1:param.N
            update_dist(param, iupdate)
        end
    end

# 原子配置の更新
    function update_config(param::Param, iupdate)
        d = 0.05*param.L
        for i in 1:3
            param.config[i,iupdate] += 2.0*d*(rand(Float64)-0.5)
            param.config[i,iupdate] = mod(param.config[i,iupdate], param.L)
        end
    end

    function update_dist(param::Param, iupdate)
        for i in 1:param.N
            tmp_dist = dist_periodic(param,param.config[:,i],param.config[:,iupdate])
            if tmp_dist > param.rcut
                param.dist[i,iupdate] = tmp_dist
                param.dist[iupdate,i] = tmp_dist
            else
                param.dist[i,iupdate] = param.rcut
                param.dist[iupdate,i] = param.rcut
            end
        end
    end

    function dist_periodic(param::Param, r1, r2)
        dist_old = sqrt(3.0)*param.L
        for iz in -1:1
            for iy in -1:1
                for ix in -1:1
                    tmp_dist = 0.0
                    tmp_dist += (param.L*ix + r1[1] - r2[1])^2
                    tmp_dist += (param.L*iy + r1[2] - r2[2])^2
                    tmp_dist += (param.L*iz + r1[3] - r2[3])^2
                    tmp_dist = sqrt(tmp_dist)
                    if tmp_dist < dist_old
                        dist_old = tmp_dist
                    end
                end
            end
        end
        return dist_old
    end

# Jastrow因子
    function func_u(param::Param, r::Float64)
        uofr = 0.0
        if r > param.rcut
            uofr = (param.a1/r)^param.a2
        else
            uofr = (param.a1/param.rcut)^param.a2
        end
        return uofr
    end

# 動径分布関数
    function accum_Nrdr(param::Param, Ndr, Nrdr)
        rmax = 0.5*param.L
        dr = rmax/Ndr
        for m in 1:Ndr
            r = rmax*(m-0.5)/Ndr
            for j in 1:(param.N-1)
                for i in (j+1):param.N
                    if (param.dist[i,j] >= r - 0.5*dr) && (param.dist[i,j] < r + 0.5*dr)
                        Nrdr[m] += 1.0
                    end
                end
            end
        end
    end
    
    function calc_gofr(param::Param, Nsample, Ndr, Nrdr, gofr, vecr)
        Ω = param.L^3
        ρ = param.N*1.0 / Ω
        rmax = 0.5*param.L
        dr = rmax/Ndr
        for m in 1:Ndr
            r = rmax*(m-0.5)/Ndr
            vecr[m] = r
            gofr[m] = Nrdr[m]/(2.0*Nsample*π*(ρ^2)*Ω*(r^2)*dr)
        end
    end

end

## モジュール、変分パラメータ、計算サイズ

In [ ]:
using .Helium4

a1 = 2.6 # 変分パラメータ
a2 = 5.0 # 変分パラメータ
L = 11.2　# 立方体周期境界条件の計算セル一辺の長さ(ang)
N = 32 # 計算セル内のヘリウム４原子の個数

param = Helium4.Param()
Helium4.initialize(param, a1, a2, L, N)
Helium4.initialize_config(param)

## 計算条件の設定

In [ ]:
# ウォーミングアップとモンテカルロ・サンプリング数、および乱数のシード
Nwup = 3200
Nsample = 12800
Random.seed!(10)
# 動径分布関数を計算するための準備
Ndr = 64
Nrdr = zeros(Float64, Ndr)
gofr = zeros(Float64, Ndr)
vecr = zeros(Float64, Ndr)

## ウォーミングアップ

In [ ]:
# warming up
config_old = zeros(Float64, 3, param.N)
dist_old = zeros(Float64, param.N, param.N)
for i in 1:Nwup
    for j in 1:param.N
        config_old .= param.config
        dist_old .= param.dist
        iupdate = Int(floor(rand(Float64)*param.N)) + 1
        if iupdate > param.N
            println("error!", iupdate)
        end
        Helium4.update_config(param,iupdate)        
        Helium4.update_dist(param,iupdate)
        lnPNt = 0.0
        lnPNi = 0.0
        for m in 1:param.N
            if m ≠ iupdate
                lnPNt -= 2.0*Helium4.func_u(param,param.dist[m,iupdate])
                lnPNi -= 2.0*Helium4.func_u(param,dist_old[m,iupdate])
                if (param.dist[m,iupdate]- dist_old[m,iupdate]) == 0.0
                   println("dist is not")
                end
            end
        end
        ratio = exp(lnPNt - lnPNi)
        if ratio <= rand(Float64)
            param.config[:,:] = config_old[:,:]
            param.dist[:,:] = dist_old[:,:]
        end
    end
    if i % 100 == 0
        println("Wup",i," ",param.config[1,1])
    end
end

## モンテカルロ・サンプリング

In [ ]:
for i in 1:Nsample
    for j in 1:param.N
        config_old .= param.config
        dist_old .= param.dist
        iupdate = Int(floor(rand(Float64)*param.N)) + 1
        Helium4.update_config(param,iupdate)
        Helium4.update_dist(param,iupdate)
        lnPNt = 0.0
        lnPNi = 0.0
        for m in 1:param.N
            if m ≠ iupdate
                lnPNt -= 2.0*Helium4.func_u(param,param.dist[m,iupdate])
                lnPNi -= 2.0*Helium4.func_u(param,dist_old[m,iupdate])
            end
        end
        ratio = exp(lnPNt - lnPNi)
        if ratio <= rand(Float64)
            param.config[:,:] = config_old[:,:]
            param.dist[:,:] = dist_old[:,:]
        end
    end
    if i % 100 == 0
        println("MCstep",i," ",param.config[1,1])
    end
    Helium4.accum_Nrdr(param,Ndr,Nrdr)
end
Helium4.calc_gofr(param,Nsample,Ndr,Nrdr,gofr,vecr)

## 計算結果のプロット

In [ ]:
plot(vecr,gofr,xlabel="r",ylabel="g(r)")

# 計算の確認

上のセル実行後に、以下のような図が表示されたら計算成功です。

[g(r)の図](https://github.com/yyamaji/many-body-electrons/blob/4dc4353067b08462159ed9fdfb9290b7290c3a9a/vmc_gofr_4He.png)